<a href="https://colab.research.google.com/github/satria-mitra/awscenter-downloader/blob/main/aws_center_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Untitled3.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1WqRD4dPAxIUKp4l1bVQe3wjolJNk1K6Z
"""

# Install required packages
!pip install requests bs4 pillow easyocr

# Importing libraries
import requests                         #pip install requests
from bs4 import BeautifulSoup as bs     #pip install bs4
from PIL import Image                   #pip install pillow
import easyocr                          #pip install easyocr
from io import BytesIO
import csv
import json
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
import os
import getpass
import csv

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the base directory for saving files
DRIVE_BASE_DIR = '/content/drive/MyDrive/Weather_Station_Data'  # You can change this path as needed
print(f"Files will be saved to: {DRIVE_BASE_DIR}")



def input_credentials():
    """Meminta user memasukkan username dan password AWS Center dengan opsi dari environment variable"""
    user = os.getenv("MY_APP_USER") or input("Masukkan AWS Center Username: ")
    passwd = os.getenv("MY_APP_PASSWD") or getpass.getpass("Masukkan AWS Center Password: ")
    return user, passwd

# Welcoming Message
print("===========================================================")
print("AWS CENTER DOWNLOADER Alternative")
print("Created by Imron Ade")
print("Modified by Satria Mitra")
print("===========================================================")

# ===========================================================
# Functions
# ===========================================================

def input_credentials():
    """Meminta user memasukkan username dan password AWS Center dengan opsi dari environment variable"""
    user = os.getenv("MY_APP_USER") or input("Masukkan AWS Center Username: ")
    passwd = os.getenv("MY_APP_PASSWD") or getpass.getpass("Masukkan AWS Center Password: ")
    return user, passwd

# Initialize OCR reader
reader = easyocr.Reader(['en','ar'], gpu=False) #by default GPU is true, if you don't have GPU, type 'False'

def bypass_captcha(url):
    res = ses.get(url)
    cap = bs(res.text, 'html.parser').find_all('img')[2]['src'] # Looking for the captcha image link

    # Downloading captcha.png
    unduh = ses.get(cap)
    im = Image.open(BytesIO(unduh.content))
    im.save("captcha.png")  # Downloading captcha images

    # Read text from captcha using EasyOCR
    result = reader.readtext("captcha.png", detail=0)
    print("Detected Captcha Text:", result)  # Print the raw OCR output

    if not result:
        return None  # Return None if no text detected

    text = result[0].replace(' ', '')  # Clean up spaces

    # Extract numbers and operator
    try:
        num1, operator, num2 = int(text[0]), text[1], int(text[2])

        if operator == 'x' or operator == '*':
            return num1 * num2
        elif operator == '+':
            return num1 + num2
        elif operator == '-':
            return num1 - num2
    except (IndexError, ValueError):
        return None  # Return None if parsing fails

    return None

# Make folder based on current date
def create_folder_structure(station_type, year, month):
    """Create the folder structure in Google Drive for a specific station type, year, and month."""
    # Create the folder structure: DRIVE_BASE_DIR/TYPE/YEAR/MONTH/
    folder_path = f'{DRIVE_BASE_DIR}/{station_type}/{year}/{month}/'

    if not os.path.isdir(folder_path):
        os.makedirs(folder_path)
        print(f'Created folder: {folder_path}')
    else:
        print(f'Folder already exists: {folder_path}')

    return folder_path

# Get label from json data
def get_label(data_json):
    label_data_json = []
    for label in data_json:
        label_data_json.append(label["label"])
    return label_data_json

# Find code from list data
def find_list_code(data_list_json, nama_list_data):
    for listing in data_list_json:
        kode_list = listing["value"]
        nama_list = listing["label"]
        if nama_list == nama_list_data:
            return kode_list
    return "Not Found"

def save_to_json(file_path, new_data):
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                existing_data = json.load(file)
        except json.JSONDecodeError:
            existing_data = []
    else:
        existing_data = []

    existing_data.extend(new_data)

    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, indent=4, ensure_ascii=False)

def load_kode_alat(file_path):
    """Load kode alat JSON."""
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

# Menu system functions
def display_menu():
    """Display the main menu options."""
    print("\n==========================================================")
    print("AWS CENTER DOWNLOADER - MENU")
    print("==========================================================")
    print("1. Download data for specific sites (Monthly or Range)")
    print("2. Download data for all sites (Monthly or Range)")
    print("3. Download data for specific custom dates (Manual Start and End)")
    print("4. Download data for yesterday only")  # <-- New Option!
    print("5. Exit")
    print("==========================================================")
    return input("Select an option (1-5): ")


def select_date_range_extended():
    """Allow user to select custom date range with multiple months."""
    print("\n==========================================================")
    print("DATE RANGE SELECTION")
    print("==========================================================")
    print("0. Go Back")
    print("1. Download a single month")
    print("2. Download a range of months")
    print("==========================================================")

    choice = input("Select an option (0-2): ")

    if choice == '0':
        return None
    elif choice == '1':
        return select_single_month()
    elif choice == '2':
        return select_month_range()
    else:
        print("Invalid choice. Please select 0-2.")
        return select_date_range_extended()

def select_single_month():
    """Select a single month and return its date range."""
    # Get year
    while True:
        year = input("Enter year (YYYY): ")
        if year.isdigit() and len(year) == 4:
            break
        print("Invalid year format. Please enter a 4-digit year.")

    # Get month
    while True:
        month = input("Enter month (MM): ")
        if month.isdigit() and 1 <= int(month) <= 12:
            # Pad with leading zero if needed
            month = month.zfill(2)
            break
        print("Invalid month. Please enter a month between 01-12.")

    # Calculate first and last day of month
    if month == '02':
        # Check for leap year
        if (int(year) % 4 == 0 and int(year) % 100 != 0) or (int(year) % 400 == 0):
            last_day = '29'
        else:
            last_day = '28'
    elif month in ['04', '06', '09', '11']:
        last_day = '30'
    else:
        last_day = '31'

    first_day = '01'

    start_date = f"{year}-{month}-{first_day}"
    end_date = f"{year}-{month}-{last_day}"

    print(f"Selected date range: {start_date} to {end_date}")
    return [(year, month, start_date, end_date)]

def select_month_range():
    """Select a range of months and return all date ranges."""
    # Get start year and month
    while True:
        start_year = input("Enter start year (YYYY): ")
        if start_year.isdigit() and len(start_year) == 4:
            break
        print("Invalid year format. Please enter a 4-digit year.")

    while True:
        start_month = input("Enter start month (MM): ")
        if start_month.isdigit() and 1 <= int(start_month) <= 12:
            # Pad with leading zero if needed
            start_month = start_month.zfill(2)
            break
        print("Invalid month. Please enter a month between 01-12.")

    # Get end year and month
    while True:
        end_year = input("Enter end year (YYYY): ")
        if end_year.isdigit() and len(end_year) == 4:
            if int(end_year) < int(start_year):
                print("End year must be greater than or equal to start year.")
                continue
            break
        print("Invalid year format. Please enter a 4-digit year.")

    while True:
        end_month = input("Enter end month (MM): ")
        if end_month.isdigit() and 1 <= int(end_month) <= 12:
            # Pad with leading zero if needed
            end_month = end_month.zfill(2)
            if int(end_year) == int(start_year) and int(end_month) < int(start_month):
                print("End month must be greater than or equal to start month for the same year.")
                continue
            break
        print("Invalid month. Please enter a month between 01-12.")

    # Generate all month ranges
    date_ranges = []
    current_year = int(start_year)
    current_month = int(start_month)
    end_year_int = int(end_year)
    end_month_int = int(end_month)

    while (current_year < end_year_int) or (current_year == end_year_int and current_month <= end_month_int):
        # Format current month with leading zero
        month_str = str(current_month).zfill(2)
        year_str = str(current_year)

        # Calculate last day of month
        if month_str == '02':
            # Check for leap year
            if (current_year % 4 == 0 and current_year % 100 != 0) or (current_year % 400 == 0):
                last_day = '29'
            else:
                last_day = '28'
        elif month_str in ['04', '06', '09', '11']:
            last_day = '30'
        else:
            last_day = '31'

        first_day = '01'

        start_date = f"{year_str}-{month_str}-{first_day}"
        end_date = f"{year_str}-{month_str}-{last_day}"

        date_ranges.append((year_str, month_str, start_date, end_date))

        # Move to next month
        current_month += 1
        if current_month > 12:
            current_month = 1
            current_year += 1

    print(f"Selected {len(date_ranges)} months from {start_year}-{start_month} to {end_year}-{end_month}")
    return date_ranges

def select_custom_date_range():
    """Allow user to input custom start and end dates manually."""
    while True:
        start_date = input("Enter start date (YYYY-MM-DD): ").strip()
        try:
            datetime.datetime.strptime(start_date, "%Y-%m-%d")
            break
        except ValueError:
            print("Invalid start date format. Please use YYYY-MM-DD.")

    while True:
        end_date = input("Enter end date (YYYY-MM-DD): ").strip()
        try:
            datetime.datetime.strptime(end_date, "%Y-%m-%d")
            if end_date < start_date:
                print("End date must be after start date.")
                continue
            break
        except ValueError:
            print("Invalid end date format. Please use YYYY-MM-DD.")

    # Extract year and month from start_date for folder naming
    year, month, _ = start_date.split('-')
    return [(year, month, start_date, end_date)]


def select_yesterday_range():
    """Automatically select yesterday's date range."""
    yesterday = date.today() - datetime.timedelta(days=1)
    start_date = yesterday.strftime("%Y-%m-%d")
    end_date = yesterday.strftime("%Y-%m-%d")
    year = yesterday.strftime("%Y")
    month = yesterday.strftime("%m")
    print(f"Selected Yesterday: {start_date}")
    return [(year, month, start_date, end_date)]

def display_sites(data_kode_alat):
    """Display available sites for selection."""
    print("\n==========================================================")
    print("AVAILABLE SITES")
    print("==========================================================")
    print("0. Go Back to Main Menu")
    print("-" * 50)

    # Group sites by type
    sites_by_type = {}
    for item in data_kode_alat:
        site_type = item['type']
        if site_type not in sites_by_type:
            sites_by_type[site_type] = []
        sites_by_type[site_type].append(item)

    # Display sites grouped by type
    site_index = 1
    site_mapping = {}

    for site_type, sites in sites_by_type.items():
        print(f"\n{site_type} SITES:")
        print("-" * 50)
        for site in sites:
            print(f"{site_index}. {site['label']} (Code: {site['value']})")
            site_mapping[site_index] = site
            site_index += 1

    return site_mapping  # Add this return statement
    """Display available sites for selection."""
    print("\n==========================================================")
    print("AVAILABLE SITES")
    print("==========================================================")
    print("0. Go Back to Main Menu")
    print("-" * 50)

    # Group sites by type
    sites_by_type = {}
    for item in data_kode_alat:
        site_type = item['type']
        if site_type not in sites_by_type:
            sites_by_type[site_type] = []
        sites_by_type[site_type].append(item)

    # Display sites grouped by type
    site_index = 1
    site_mapping = {}

    for site_type, sites in sites_by_type.items():
        print(f"\n{site_type} SITES:")
        print("-" * 50)
        for site in sites:
            print(f"{site_index}. {site['label']} (Code: {site['value']})")
            site_mapping[site_index] = site
            site_index += 1

def select_sites(site_mapping):
    """Allow user to select specific sites."""
    selected_sites = []

    while True:
        print("\nEnter site numbers to download (comma-separated), '0' to go back, or 'all' for all sites:")
        selection = input("> ").strip().lower()

        if selection == '0':
            return None  # Signal to go back to main menu

        if selection == 'all':
            return list(site_mapping.values())

        try:
            # Parse comma-separated site numbers
            site_numbers = [int(num.strip()) for num in selection.split(',')]

            # Check for '0' in the selection
            if 0 in site_numbers:
                return None  # Signal to go back to main menu

            # Validate site numbers
            invalid_numbers = [num for num in site_numbers if num not in site_mapping]
            if invalid_numbers:
                print(f"Invalid site numbers: {', '.join(map(str, invalid_numbers))}")
                continue

            # Get selected sites
            selected_sites = [site_mapping[num] for num in site_numbers]
            break
        except ValueError:
            print("Invalid input. Please enter comma-separated numbers, '0' to go back, or 'all'.")

    return selected_sites

    """Allow user to select specific sites."""
    selected_sites = []

    while True:
        print("\nEnter site numbers to download (comma-separated), or 'all' for all sites:")
        selection = input("> ").strip().lower()

        if selection == 'all':
            return list(site_mapping.values())

        try:
            # Parse comma-separated site numbers
            site_numbers = [int(num.strip()) for num in selection.split(',')]

            # Validate site numbers
            invalid_numbers = [num for num in site_numbers if num not in site_mapping]
            if invalid_numbers:
                print(f"Invalid site numbers: {', '.join(map(str, invalid_numbers))}")
                continue

            # Get selected sites
            selected_sites = [site_mapping[num] for num in site_numbers]
            break
        except ValueError:
            print("Invalid input. Please enter comma-separated numbers or 'all'.")

    return selected_sites

def download_data_extended(selected_sites, date_ranges, cookies_dictionary):
    """Download data for selected sites across multiple date ranges."""
    total_downloads = len(selected_sites) * len(date_ranges)

    print("\n==========================================================")
    print(f"DOWNLOADING DATA FOR {len(selected_sites)} SITES ACROSS {len(date_ranges)} MONTHS")
    print("==========================================================")

    download_count = 0

    for date_range in date_ranges:
        year, month, start_date, end_date = date_range

        # Download data for each selected site
        for site in selected_sites:
            download_count += 1
            site_type = site['type']
            site_code = site['value']
            site_name = site['label']

            print(f"[{download_count}/{total_downloads}] Downloading {site_name} ({site_type}) for {year}-{month}...")

            # Get type code from listTipe
            type_code = listTipe.get(site_type)
            if not type_code:
                print(f"Error: Type code not found for {site_type}")
                continue

            # Create folder structure
            folder_path = create_folder_structure(site_type, year, month)

            url = 'https://awscenter.bmkg.go.id/accessdata/bridge_to_api'
            body = {
                'tipe_alat': type_code,
                'start': start_date,
                'end': end_date,
                'station': site_code
            }

            try:
                r = requests.post(url, cookies=cookies_dictionary, data=body)
                r.raise_for_status()

                raw_html = r.text
                soup = bs(raw_html, "lxml")
                headers = [th.string for th in soup.find_all('th')]
                trs = soup.find_all('tr')

                # Format the station name for the filename (lowercase, replace spaces with underscores)
                formatted_name = site_name.lower().replace(' ', '_')

                # Save file with the format: YYYY_MM_stationname.csv (without redundant type)
                output_path = f'{folder_path}{year}_{month}_{formatted_name}.csv'

                with open(output_path, 'w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(headers)
                    for tr in trs[::-1]:
                        writer.writerow([td.string for td in tr.find_all('td')])

                print(f"✓ Successfully downloaded {site_name} data for {year}-{month}")
                print(f"  Saved to: {output_path}")
            except Exception as e:
                print(f"✗ Error downloading {site_name} data for {year}-{month}: {str(e)}")

# ===========================================================
# Main execution
# ===========================================================

# User and password
user, passwd = input_credentials()

print(f"Username yang digunakan: {user}")
print(f"Password yang digunakan: {'*' * len(passwd)}")

# Setting up date
bulan = date.today() - relativedelta(months=1)
bulanDate = bulan.replace(day=1)
awal = bulanDate.strftime('%d')
last_month = date.today().replace(day=1) - datetime.timedelta(days=1)
akhir = last_month.strftime('%d')
tahun = bulanDate.strftime('%Y')
bulan = bulanDate.strftime('%m')

tanggalAwal = f'{tahun}-{bulan}-{awal}'
tanggalAkhir = f'{tahun}-{bulan}-{akhir}'

# Login session
print("Login Session")
ses = requests.session()

# Login session
while True:
    url1 = 'https://awscenter.bmkg.go.id/base'
    # Process of searching for captcha answer
    try:
        hasil = bypass_captcha(url1)

        url2 = 'https://awscenter.bmkg.go.id/base/verify'
        data = {
            'username': user,
            'password': passwd,
            'captcha': hasil
        }
        res2 = ses.post(url2, data=data)

        if "Welcome" not in res2.text:
            print("Login tidak berhasil!")
            print("Sedang mencoba kembali untuk login")
            continue
    except:
        print("Login tidak berhasil!")
        print("Sedang mencoba kembali untuk login")
        continue
    break

print("Berhasil Login")

# Get ci_session
ses_cookies = ses.cookies
cookies_dictionary = ses_cookies.get_dict()

# Load list of type
url = "https://awscenter.bmkg.go.id/base/autocomplete"
body = {
    "act": "data_type"
}
r = requests.post(url, cookies=cookies_dictionary, json=body)
my_json = r.content.decode('utf8').replace("'", '"')
data = json.loads(my_json)
data_json = data

tipeAlat = {}
labelAlat = get_label(data_json)

for i in labelAlat:
    kodeAlat = find_list_code(data_json, i)
    if kodeAlat:
        tipeAlat[i] = kodeAlat

kodeAlat = {}
for label in labelAlat:
    kode = find_list_code(data, label)
    if kode:
        kodeAlat[label] = kode

mustType = ['AWS', 'AAWS', 'ARG']
listTipe = {}

for i in mustType:
    kode = tipeAlat.get(i, None)
    if kode:
        listTipe[i] = kode

# Generate kode_alat.json if it doesn't exist
json_path = f"{DRIVE_BASE_DIR}/kode_alat.json"
if not os.path.exists(json_path):
    # Make sure the base directory exists
    if not os.path.exists(DRIVE_BASE_DIR):
        os.makedirs(DRIVE_BASE_DIR)

    for i in listTipe:
        tipe = listTipe[i]
        url = 'https://awscenter.bmkg.go.id/base/autocomplete'
        body = {
            "act": "data_stasiun_assigned_bytype",
            "type": tipe
        }

        try:
            r = requests.post(url, cookies=cookies_dictionary, json=body)
            r.raise_for_status()
        except requests.RequestException:
            continue

        my_json = r.content.decode('utf8').replace("'", '"')
        data = json.loads(my_json)

        labelAlat = get_label(data)

        kodeAlat = {}
        for label in labelAlat:
            kode = find_list_code(data, label)
            kodeAlat[label] = kode

        final_data = []

        for label, kode in kodeAlat.items():
            entry = {
                "value": kode,
                "type": i,
                "label": label
            }
            final_data.append(entry)

        save_to_json(json_path, final_data)

# Main function
def main():
    # Load kode alat from JSON
    json_path = f"{DRIVE_BASE_DIR}/kode_alat.json"
    try:
        data_kode_alat = load_kode_alat(json_path)
    except FileNotFoundError:
        print(f"Error: {json_path} not found. Please run the code to generate it first.")
        return

    # Main menu loop
    while True:
        choice = display_menu()

        if choice == '1':  # Download specific sites (Monthly/Range)
            while True:
                site_mapping = display_sites(data_kode_alat)
                selected_sites = select_sites(site_mapping)

                if selected_sites is None:
                    break

                if selected_sites:
                    date_ranges = select_date_range_extended()
                    if date_ranges is None:
                        continue
                    download_data_extended(selected_sites, date_ranges, cookies_dictionary)
                    break
                else:
                    print("No sites selected.")

        elif choice == '2':  # Download all sites (Monthly/Range)
            while True:
                date_ranges = select_date_range_extended()
                if date_ranges is None:
                    break
                download_data_extended(data_kode_alat, date_ranges, cookies_dictionary)
                break

        elif choice == '3':  # Manual custom dates
            while True:
                site_mapping = display_sites(data_kode_alat)
                selected_sites = select_sites(site_mapping)

                if selected_sites is None:
                    break

                if selected_sites:
                    date_ranges = select_custom_date_range()
                    if date_ranges is None:
                        continue
                    download_data_extended(selected_sites, date_ranges, cookies_dictionary)
                    break
                else:
                    print("No sites selected.")

        elif choice == '4':  # Download yesterday only
            while True:
                site_mapping = display_sites(data_kode_alat)
                selected_sites = select_sites(site_mapping)

                if selected_sites is None:
                    break

                if selected_sites:
                    date_ranges = select_yesterday_range()
                    download_data_extended(selected_sites, date_ranges, cookies_dictionary)
                    break
                else:
                    print("No sites selected.")

        elif choice == '5':  # Exit
            print("Exiting program. Goodbye!")
            break

        else:
            print("Invalid choice. Please select 1-5.")


# Run the main function
if __name__ == "__main__" or 'google.colab' in str(get_ipython()):
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files will be saved to: /content/drive/MyDrive/Weather_Station_Data
AWS CENTER DOWNLOADER Alternative
Created by Imron Ade
Modified by Satria Mitra
Masukkan AWS Center Username: 96107
Masukkan AWS Center Password: ··········
Username yang digunakan: 96107
Password yang digunakan: **********
Login Session
Detected Captcha Text: ['4-7= ?']
Berhasil Login

AWS CENTER DOWNLOADER - MENU
1. Download data for specific sites (Monthly or Range)
2. Download data for all sites (Monthly or Range)
3. Download data for specific custom dates (Manual Start and End)
4. Download data for yesterday only
5. Exit
